In [ ]:
# https://blog.quantinsti.com/asset-beta-market-beta-python/

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

from pandas_datareader import data as pdr
import yfinance as yf

import statsmodels.api as sm
from statsmodels import regression

In [ ]:
df_full = pd.read_csv('data/prices-split-adjusted.csv')
yf.pdr_override()
# 这堆stocks网上抄的code run不出来 生气
notoklist = ["ARNC","APC","BBT","BCR","BHI","CAH","CAT","CBG","CMCSA",
             "COH","DOW","DPS","FOX","FOXA","GGP","GPS","HCN","HCP",
            "HRS","IR","LLL","LLTC","LUK","LVLT","MJN","PCLN","PX","RAI",
            "RHT","RTN","SE","SPLS","SYMC","TMK","TSO","TSS","WFM",
            "WYN","YHOO","DLPH","KORS"]

In [ ]:
def linreg(x,y):
    x = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x).fit()
    
    x = x[:,1]
    return model.params[0], model.params[1]
def getBeta(X,Y):
    alpha, beta = linreg(X,Y)
#     print('alpha: ' + str(alpha))
#     print('beta: ' + str(beta))
    return beta
def get_data_stock(stock):
    df = df_full[(df_full['symbol'] == stock)][['date','close']]
    df = df[df['date'] >= "2016-01-01"]
    df = df[df['date'] <= "2016-12-31"]
    df = df.rename(columns={'close':'Close'})
    df.set_index('date',inplace=True)
    return df

In [ ]:
# start = "2016-01-01", end="2016-12-30"
beta_list = []
def chartnbeta(stock,start,end):
    if stock in notoklist:
        df1 = get_data_stock(stock)
    else:
        df1 = pdr.get_data_yahoo(stock, start=start, end = end)
    df2 = pdr.get_data_yahoo("SPY", start=start, end = end)
    df2.drop(columns='Close',inplace=True)
    df2 = df2.rename(columns={'Adj Close':'Close'})
    return_aapl = df1.Close.pct_change()[1:]
    return_spy = df2.Close.pct_change()[1:]
    
    X = return_spy.values
    Y = return_aapl.values
    
    beta = getBeta(X,Y)
    beta_list.append([stock,beta])

In [ ]:
stocks = df_full['symbol'].unique()
for stock in stocks:
    chartnbeta(stock,start = "2016-01-01", end="2016-12-31")
print('DONE!')

In [ ]:
beta_list

In [ ]:
# FTV
a = df_full[df_full['symbol'] == "FTV"]
a = a[a['date'] >= "2016-01-01"]
a = a[a['date'] <= "2016-12-31"]
a = a.rename(columns={'close':'Close'})
a.set_index('date',inplace=True)
df1 = a
df2 = pdr.get_data_yahoo("SPY", start="2016-07-05", end = "2016-12-31")
df2.drop(columns='Close',inplace=True)
df2 = df2.rename(columns={'Adj Close':'Close'})
return_aapl = df1.Close.pct_change()[1:]
return_spy = df2.Close.pct_change()[1:]
    
X = return_spy.values
Y = return_aapl.values
    
beta = getBeta(X,Y)
print(["FTV",beta])

In [ ]:
df = pd.DataFrame(beta_list,columns = ['symbol','beta'])

In [ ]:
df = df.append({'symbol':'FTV','beta':1.0973156807470907},ignore_index=True)

In [ ]:
df.to_csv('2016_beta.csv',index=False)